# Notebook for making comparison between my solvers and Yandex solver

In [1]:
from lib.solver_tools.solver_client import SolverClient
from lib.solver_tools import Task
from lib.solver_tools.utils import make_task_comparison_link
import json
from copy import deepcopy
from lib.common_tools.read_token import read_solver_apikey
import os

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import vrp_algorithms_lib.common_tools.file_utils as file_utils

In [3]:
my_apikey = read_solver_apikey()
solver_client = SolverClient(my_apikey)

In [4]:
request_path = '../test_data/inputs/large_test_2/request.json'
request = file_utils.read_json(request_path)

In [5]:
task_id = solver_client.add_task_and_get_task_id(request)

/b2bgeo/environments/common_env/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'courier.yandex.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
Task(task_id).map()

Neither solver_api_key nor ya_courier_token provided to SolverClient. Using client in limited functionality mode


In [18]:
def get_request_with_fixed_routes(request, routes):
    # routes must contain fields 'vehicle_id', 'location_ids'
    new_request = deepcopy(request)
    vehicle_id_to_location_ids = {route['vehicle_id'] : route['location_ids'] for route in routes}
    for vehicle in new_request['vehicles']:
        vehicle['visited_locations'] = [{'id': location_id} for location_id in vehicle_id_to_location_ids[vehicle['id']]]
    return new_request

In [19]:
routes = file_utils.read_json('../test_data/results/annealing/large_test_2/2000000_iterations_routes.json')

In [20]:
request_with_fixed_routes = get_request_with_fixed_routes(request, routes['routes'])

In [21]:
my_task_id = solver_client.add_task_and_get_task_id(request_with_fixed_routes)

/b2bgeo/environments/common_env/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'courier.yandex.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [34]:
my_task_id_2 = solver_client.add_task_and_get_task_id(get_request_with_fixed_routes(request, file_utils.read_json('../test_data/results/annealing/large_test_2/30000000_iterations_routes.json')['routes']))

/b2bgeo/environments/common_env/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'courier.yandex.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [36]:
Task(my_task_id_2).map()

Neither solver_api_key nor ya_courier_token provided to SolverClient. Using client in limited functionality mode


ValueError: Task 73d9f103-13f07ee8-f592318d-951fe8d hasn't been solved yet. ETA 01:48:46

In [35]:
make_task_comparison_link([task_id, my_task_id, my_task_id_2])